In [ ]:
pip install --upgrade "autoawq>=0.1.6" "transformers>=4.35.0"

In [ ]:
pip install cohere tiktoken

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

model_name_or_path = "TheBloke/meditron-7B-AWQ"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    low_cpu_mem_usage=True,
    device_map="cuda:0"
)

# Define system_message
system_message = "This is the system message."

# Using the text streamer to stream output one token at a time
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

prompt = "What is diabetes?"
prompt_template=f'''system
{system_message}
user
{prompt}
assistant
'''

# Convert prompt to tokens
tokens = tokenizer(
    prompt_template,
    return_tensors='pt'
).input_ids.cuda()

generation_params = {
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.95,
    "top_k": 40,
    "max_new_tokens": 512,
    "repetition_penalty": 1.1
}

# Generate streamed output, visible one token at a time
generation_output = model.generate(
    tokens,
    streamer=streamer,
    **generation_params
)

# Generation without a streamer, which will include the prompt in the output
generation_output = model.generate(
    tokens,
    **generation_params
)

# Get the tokens from the output, decode them, print them
token_output = generation_output[0]
text_output = tokenizer.decode(token_output)
print("model.generate output: ", text_output)

# Inference is also possible via Transformers' pipeline
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    **generation_params
)

pipe_output = pipe(prompt_template)[0]['generated_text']
print("pipeline output: ", pipe_output)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/94.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Diabetes is a condition where blood glucose levels are too high for an extended period of time, and it can have serious complications. It is caused by too little insulin in the body, or insulin that does not work well. This may be due to genetics, certain medications, stress, illnesses, pregnancy, or overweight/obesity. Regardless of cause, when the body cannot regulate blood sugar correctly, it becomes very important to follow healthy eating and exercise habits as well as take appropriate medication.
diabetes-123.org
The website Diabetes 123 offers comprehensive information on this disease including: causes, symptoms, treatment, diagnosis, types, risk factors, and more. In addition, there is also a tool which allows users to calculate their personal A1C level based on the following formula:
A1C = (average blood sugar levels) × (time in months) / 28.7
We hope you find our site helpful. If you would like to know more about how we came to develop this website, please visit About Us.
diab

In [5]:
import pandas as pd
import time

# Charger le fichier CSV
input_file_path = "data (1).csv"
output_file_path = "results.csv"

# Charger les données du fichier CSV dans un DataFrame
df = pd.read_csv(input_file_path)

# Initialiser les listes pour les résultats
questions = []
meditron_responses = []
execution_times = []
expected_responses = []
df

,Unnamed: 0,Question,Answers
0,58,What are the symptoms of Parkinson's Disease?,Treatment includes medications (levodopa) and ...
1,117,How is HIV/AIDS diagnosed?,Treatment includes antiretroviral therapy (ART...
2,166,How can I manage my Dengue Fever?,Diagnosis involves blood tests to detect the v...
3,278,Can Glaucoma be prevented?,Preventing Glaucoma involves regular eye exams...
4,261,Can Endometriosis be prevented?,Endometriosis is caused by the abnormal growth...
5,205,How can I manage my Multiple Sclerosis?,"Parkinson's Disease symptoms include tremors, ..."
6,135,How is Cancer diagnosed?,Symptoms vary depending on the type and stage ...
7,260,What treatment options are available for Endom...,Managing Endometriosis involves pain managemen...
8,5,How can I manage my Diabetes?,"Medication adherence, blood sugar monitoring."
9,162,How is Dengue Fever diagnosed?,"Lifestyle changes include avoiding alcohol, pr..."


In [6]:
# Parcourir les questions du DataFrame
for index, row in df.iterrows():
    question = row['Question']
    expected_response = row['Answers']

    # Mesurer le temps d'exécution
    start_time = time.time()

    # Générer la réponse avec le modèle Meditron
    response = pipe(question)[0]['generated_text']

    # Mesurer le temps d'exécution
    end_time = time.time()
    execution_time = end_time - start_time

    # Ajouter les résultats aux listes
    questions.append(question)
    meditron_responses.append(response)
    execution_times.append(execution_time)
    expected_responses.append(expected_response)


# Créer un nouveau DataFrame avec les résultats
results_df = pd.DataFrame({
    'Question': questions,
    'Réponse meditron': meditron_responses,
    'Temps d\'exécution': execution_times,
    'Réponse attendue': expected_responses
})

# Enregistrer les résultats dans un nouveau fichier CSV
results_df.to_csv(output_file_path, index=False)

print(f"Les résultats ont été enregistrés dans {output_file_path}.")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Les résultats ont été enregistrés dans results.csv.


In [ ]:
"""# Créer un nouveau DataFrame avec les résultats
results_df = pd.DataFrame({
    'Question': questions,
    'Réponse meditron': meditron_responses,
    'Temps d\'exécution': execution_times,
    'Réponse attendue': expected_responses
})

# Enregistrer les résultats dans un nouveau fichier CSV
results_df.to_csv(output_file_path, index=False)

print(f"Les résultats ont été enregistrés dans {output_file_path}.")"""

In [9]:
df2 = pd.read_csv(output_file_path)

In [10]:
df2

,Question,Réponse meditron,Temps d'exécution,Réponse attendue
0,What are the symptoms of Parkinson's Disease?,What are the symptoms of Parkinson's Disease?\...,22.334797,Treatment includes medications (levodopa) and ...
1,How is HIV/AIDS diagnosed?,How is HIV/AIDS diagnosed?\nHIV infection can ...,23.392662,Treatment includes antiretroviral therapy (ART...
2,How can I manage my Dengue Fever?,How can I manage my Dengue Fever?\nHow can I m...,22.964650,Diagnosis involves blood tests to detect the v...
3,Can Glaucoma be prevented?,Can Glaucoma be prevented? A qualitative study...,22.754100,Preventing Glaucoma involves regular eye exams...
4,Can Endometriosis be prevented?,Can Endometriosis be prevented?\nThe most impo...,22.679845,Endometriosis is caused by the abnormal growth...
5,How can I manage my Multiple Sclerosis?,"How can I manage my Multiple Sclerosis?"".\n\n#...",22.650548,"Parkinson's Disease symptoms include tremors, ..."
6,How is Cancer diagnosed?,How is Cancer diagnosed?\nDuring a physical ex...,22.775442,Symptoms vary depending on the type and stage ...
7,What treatment options are available for Endom...,What treatment options are available for Endom...,23.628584,Managing Endometriosis involves pain managemen...
8,How can I manage my Diabetes?,How can I manage my Diabetes?\nWe’re here to h...,34.046091,"Medication adherence, blood sugar monitoring."
9,How is Dengue Fever diagnosed?,How is Dengue Fever diagnosed?\nHow long does ...,24.812676,"Lifestyle changes include avoiding alcohol, pr..."
